In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# LangGraph Tools Intro

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o")
small_llm = ChatOpenAI(model_name="gpt-4o-mini")


In [4]:
from langchain_core.tools import tool

# Required: tool decorator, expected arguments, description
@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다"""
    return a * b

In [5]:
add.invoke({"a": 1, "b": 2})

3

In [9]:
from langgraph.prebuilt import ToolNode

tool_list = [add, multiply]
llm_with_tools = small_llm.bind_tools(tool_list)
# AnyMessage(System, Human, AI, Tool) 리스트를 입력으로 받아야한다.
# 마지막은 AI message 이어야하고, 해당 메시지에 tool_calls가 있어야한다.
tool_node = ToolNode(tool_list)

In [7]:
ai_message = llm_with_tools.invoke("Hwat is 3 plus 5 ?")
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_NsatMp4aLDCGBD3sis9d8XdE', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 85, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-C3AcT3Xs1av2vLPyF6lbSKcNL8ibR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--36955819-5abe-462b-a029-41fcb5768d11-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_NsatMp4aLDCGBD3sis9d8XdE', 'type': 'tool_call'}], usage_metadata={'input_tokens': 85, 'output_tokens': 17, 'total_tokens': 102, 'input_token_details': {'audio': 0, 'cache_read':

In [10]:
tool_node.invoke({"messages": [ai_message]}) # message -> list[AnyMessage]

{'messages': [ToolMessage(content='8', name='add', tool_call_id='call_NsatMp4aLDCGBD3sis9d8XdE')]}

In [20]:
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)


In [14]:
def agent(state: MessagesState):
    message = state["messages"]
    response = llm_with_tools.invoke(message)
    return {"messages": [response]}

In [19]:
# Agent는 tool을 사용할 필요가 없을 때 메시지에 tool_calls가 없고, content에 답변이 들어가게된다 는 점을 이용

from langgraph.graph import END
def should_continue(state: MessagesState):
    message = state["messages"]
    last_ai_message = message[-1]
    if last_ai_message.tool_calls:
        return 'tools'
    return END

In [21]:
graph_builder.add_node('agent', agent)
graph_builder.add_node('tools', tool_node)


In [22]:
from langgraph.graph import START, END

graph_builder.add_edge(START, 'agent')
graph_builder.add_conditional_edges("agent", should_continue, ["tools", END])
graph_builder.add_edge("tools", "agent")



In [24]:
graph = graph_builder.compile()

In [ ]:
from IPython.display import display, Image
display(Image(graph.get_graph().draw_mermaid_png()))

ImportError: cannot import name 'Image' from 'IPython' (/home/luke/Study/langchain/kang/.venv/lib/python3.13/site-packages/IPython/__init__.py)